In [11]:
import json
import pandas as pd
import gensim.downloader as api
from sklearn.utils import shuffle
import zipfile
import sys
import requests, io
import re 
import random
import numpy as np
import matplotlib.pyplot as plt

import json
import pandas as pd
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim.models as models
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models import KeyedVectors
import gensim.downloader as api
from sklearn.utils import shuffle
import zipfile
import sys
import requests, io
import re 
import random
import numpy as np
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
def preprocess(text:str, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if re.match(r'(\d.|\d)', lemma) is None:
                if lemma not in stop_words:
                    preprocessed_text.append(lemma)
    return preprocessed_text

def read_json(path: str):
    file = open(path)
    data = json.load(file)
    return pd.DataFrame(data)

def get_states(sim, df, match_threshold):
    (TP, FP, FN, TN) = (0, 0, 0, 0)
    for i in range(sim.size):
        if df['need_match'][i]:
            if sim[i] >= match_threshold: 
                TP += 1
            else:
                FN += 1
        else:
            if sim[i] >= match_threshold: 
                FP += 1
            else:
                TN += 1
    
    return (TP, FP, FN, TN)
        

def calc_f1_score(sim, df, match_threshold):
    (TP, FP, FN, TN) = get_states(sim, df, match_threshold)
#     print(TP, FP, FN, TN)
    return round(float(2*TP / (2*TP + FP + FN)), 3)

def calc_accuracy(sim, df, match_threshold):
    (TP, FP, FN, TN) = get_states(sim, df, match_threshold)
    return round(float((TP + TN) / (TP + TN + FP + FN)), 3)

class ModelResearcher:
    def __init__(self):
        self.model = None
    
    def load_transformer(self, name):
        self.model = SentenceTransformer.load(name)
    
    def preprocess_and_save(self, data_df: pd.DataFrame, path, text_field='text') -> pd.DataFrame:
        # for preprocessing dataset. Use it only in critical cases cause it's too slow on big datasets
        data_df['preprocessed_' + text_field] = data_df.apply(lambda row: preprocess(row[text_field], punctuation_marks, stop_words, morph), axis=1)
        data_df_preprocessed = data_df.copy()
        data_df_preprocessed = data_df_preprocessed.drop(columns=[text_field], axis=1)
        data_df_preprocessed.reset_index(drop=True, inplace=True)
        if path is not None:
            data_df_preprocessed.to_json(path)
        return data_df_preprocessed
    
    def preprocess_and_save_pairs(self, data_df: pd.DataFrame, path, text_field_1, text_field_2) -> pd.DataFrame:
        data_df['preprocessed_' + text_field_1] = data_df.apply(lambda row: preprocess(row[text_field_1], punctuation_marks, stop_words, morph), axis=1)
        data_df['preprocessed_' + text_field_2] = data_df.apply(lambda row: preprocess(row[text_field_2], punctuation_marks, stop_words, morph), axis=1)
        data_df_preprocessed = data_df.copy()
        data_df_preprocessed = data_df_preprocessed.drop(columns=[text_field_1, text_field_2], axis=1)
        data_df_preprocessed.reset_index(drop=True, inplace=True)
        if path is not None:
            data_df_preprocessed.to_json(path)
        return data_df_preprocessed
    
    def train(self, data_df: pd.DataFrame, model="w2v"):            
        if model == "w2v":
            train_part = data_df['preprocessed_texts']
            self.model = gensim.models.Word2Vec(sentences=train_part, min_count=5, vector_size=50, epochs=5)
            self.model.save('w2v')
        elif model == "fast_text":
            train_part = data_df['preprocessed_texts'].tolist()
            self.model = gensim.models.FastText(vector_size=50, min_count=5)
            self.model.build_vocab(corpus_iterable=train_part)
            self.model.train(corpus_iterable=train_part, total_examples=len(train_part), epochs=5)
            self.model.save('fastText')
        return

                    
    
    def predict_sentences_similarity(self, sentences_1: pd.Series, sentences_2: pd.Series):
        if sentences_1.size != sentences_2.size:
            return None
        else:
            if self.model is not None:
                sentences_sim = np.zeros(sentences_1.size)
                sz = sentences_1.size
                for i in range(sz): 
                    sentences_1_words = [w for w in sentences_1[i] if w in self.model.wv.index_to_key]
                    sentences_2_words = [w for w in sentences_2[i] if w in self.model.wv.index_to_key]
                    sim = self.model.wv.n_similarity(sentences_1_words, sentences_2_words)
                    sentences_sim[i] = sim
                
                return sentences_sim
            else:
                return None
        
    def get_train_test_dfs_for_f1(self, df_match, df_unmatch): 
        df_match = shuffle(df_match)
        df_unmatch = shuffle(df_unmatch)
        
        df_train_f1 = pd.concat([pd.DataFrame(df_match[0:len(df_match) // 2]), pd.DataFrame(df_unmatch[0:len(df_unmatch) // 2])], axis=0)
        df_test_f1 = pd.concat([pd.DataFrame(df_match[len(df_match) // 2 : ]), pd.DataFrame(df_unmatch[len(df_unmatch) // 2 : ])], axis=0)
        return (df_train_f1, df_test_f1)
    
    
    def get_optimal_threshold(self, sentences_1: pd.Series, sentences_2: pd.Series, df, step=0.02):
        if sentences_1.size != sentences_2.size or self.model is None:
            return None
        else:
            threshold = 0
            thresholds = []
            max_ = 0
            h = step
            steps = np.linspace(0, 1, num=int(1/h))
            steps = np.round(steps, 2)
            for i in steps:
                sim =  self.predict_sentences_similarity( sentences_1, sentences_2)
                threshold = calc_f1_score(sim, df, h)
                print(h, threshold)
                thresholds.append(threshold)
                if threshold > max_:
                    max_ = threshold
                h += step
            plt.plot(steps, thresholds)
            plt.figure(figsize=(10,3))
            plt.show()
            
            return max_

punctuation_marks = ['!', ',', '(', ')', ';', ':', '-', '?', '.', '..', '...', "\"", "/", "\`\`", "»", "«" ]
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [2]:
from sentence_transformers import SentenceTransformer

In [6]:
model = SentenceTransformer('DeepPavlov/rubert-base-cased')

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

No sentence-transformers model found with name C:\Users\Andrey/.cache\torch\sentence_transformers\DeepPavlov_rubert-base-cased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\Andrey/.cache\torch\sentence_transformers\DeepPavlov_rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly id

In [22]:
# model.save('./models/Pavlov-rubert')

In [23]:
model = SentenceTransformer.load('./models/Pavlov-rubert')

In [31]:
match_unmatch_df = read_json('./dataset.json')

In [32]:
match_unmatch_df

,id_rp,id_proj,name_rp,name_proj,text_rp,text_proj,need_match
0,1000,15,АЛГОРИТМЫ БЕСПИЛОТНОГО ТРАНСПОРТА,Разработка системы движения роя дронов в прост...,Данный курс знакомит слушателей с основами упр...,Разработка системы движения роя дронов в прост...,True
1,1005,40,АНАЛИЗ ДАННЫХ В ИСКУССТВЕННОМ ИНТЕЛЛЕКТЕ,Роботизированная система машинного творчества ...,Освещаемые в курсе теоретические и прикладные ...,Роботизированная система машинного творчества ...,True
2,1009,4,РАЗРАБОТКА ПРИЛОЖЕНИЙ В РАСПРЕДЕЛЕННОЙ СРЕДЕ,Разработка модулей обработки данных для библио...,Дисциплина посвящена изучению основных принцип...,Разработка модулей обработки данных для библио...,True
3,1011,14,Тестирование и сопровождение программного обес...,Тестирование и документирование стенда виртуал...,"Цели дисциплины охватывают ряд направлений, св...",Тестирование и документирование стенда виртуал...,True
4,1016,46,Интеллектуальные системы,Разработка интеллектуальных модулей обработки ...,Рассматриваются основные понятия теории интелл...,Разработка интеллектуальных модулей обработки ...,True
5,105,41,Системы управления манипуляторными роботами,Разработка подводного телеуправляемого робота ...,Дисциплина включает изложение основ теории ман...,Разработка подводного телеуправляемого робота ...,True
6,1040,26,Управление и информатика в технических системах,Программно-аппаратное конфигурирование сети ла...,В рамках данной дисциплины рассматриваются осн...,Программно-аппаратное конфигурирование сети ла...,True
7,1278,9,Тестирование программного обеспечения,Тестирование конфигураций на платформах: Super...,Дисциплина посвящена изучению теоретических ос...,Тестирование конфигураций на платформах: Super...,True
8,991,30,Машинное обучение,Разработка системы детектирования и идентифика...,Данный курс знакомит слушателей с основными ме...,Разработка системы детектирования и идентифика...,True
9,985,8,Основы компьютерного дизайна,"Разработка ИС ""Расписание"" / ""Деканат"".",В курсе рассматриваются свойства зрительного в...,"Разработка ИС ""Расписание"" / ""Деканат"". Информ...",True


In [33]:
df_match = match_unmatch_df[ match_unmatch_df['need_match'] == True][:-2]
df_unmatch = match_unmatch_df[match_unmatch_df['need_match'] == False]

modelResearcher_rubert = ModelResearcher()
modelResearcher_rubert.load_transformer('./models/Pavlov-rubert')
df_train_f1, df_test_f1 = modelResearcher_rubert.get_train_test_dfs_for_f1(df_match, df_unmatch)

In [34]:
print('df_train: {} of {}'.format(df_train_f1['id_rp'].size, match_unmatch_df ['id_rp'].size))
print('df_test: {} of {}'.format(df_test_f1['id_rp'].size,match_unmatch_df['id_rp'].size))

df_train: 22 of 46
df_test: 22 of 46


In [39]:
# df_train_f1['text_rp']
# df_train_f1['text_proj']
sentence_embeddings_rp = []
sentence_embeddings_proj = []
for item in df_train_f1['text_rp']:
    sentence_embeddings_rp.append(modelResearcher_rubert.model.encode(item, convert_to_tensor=True))

for item in df_train_f1['text_proj']:
    sentence_embeddings_proj.append(modelResearcher_rubert.model.encode(item, convert_to_tensor=True))
    

#     util.pytorch_cos_sim(sentence_embeddings_rp, embedding_2)

[tensor([ 1.2091e-01, -4.2185e-01,  7.5422e-01, -3.5474e-01,  2.2741e-01,
         2.2983e-01, -1.2966e-01,  6.7871e-01, -2.8404e-01, -1.7096e-01,
         1.8409e-01,  1.2977e-02, -2.3902e-01,  1.6588e-01,  3.9187e-02,
        -4.7163e-01,  3.0814e-01, -1.1318e-01, -2.2067e-01,  2.1776e-01,
        -1.4425e-01,  2.7749e-01,  1.7485e-01, -1.0968e-01,  5.9227e-02,
         5.0555e-01, -4.4753e-01, -3.2827e-01, -2.6525e-01, -1.0701e-02,
         1.8206e-01,  2.0775e-02, -2.4985e-01,  7.0956e-03,  7.0346e-02,
         2.5579e-01,  3.4013e-01,  1.2085e-01, -3.6209e-01,  3.4075e-01,
        -6.5035e-02,  1.0430e-01,  4.8675e-01, -1.7171e-01, -5.7619e-02,
         1.0693e-01, -7.5097e-02,  5.4318e-02, -4.1089e-01, -4.0388e-01,
        -8.5236e-02, -1.2121e-02,  1.1091e-01, -2.9335e-01,  1.5991e-01,
         9.0246e-03,  1.4291e-01, -9.6933e-02,  2.6828e-01,  2.8086e-01,
         1.6056e-01,  2.6656e-01, -2.4157e-01, -1.3992e-01,  1.3189e-01,
         4.3679e-01, -2.7167e-02, -3.2675e-01, -1.

In [40]:
print(sentence_embeddings_rp)
print(sentence_embeddings_proj)

[tensor([ 1.2091e-01, -4.2185e-01,  7.5422e-01, -3.5474e-01,  2.2741e-01,
         2.2983e-01, -1.2966e-01,  6.7871e-01, -2.8404e-01, -1.7096e-01,
         1.8409e-01,  1.2977e-02, -2.3902e-01,  1.6588e-01,  3.9187e-02,
        -4.7163e-01,  3.0814e-01, -1.1318e-01, -2.2067e-01,  2.1776e-01,
        -1.4425e-01,  2.7749e-01,  1.7485e-01, -1.0968e-01,  5.9227e-02,
         5.0555e-01, -4.4753e-01, -3.2827e-01, -2.6525e-01, -1.0701e-02,
         1.8206e-01,  2.0775e-02, -2.4985e-01,  7.0956e-03,  7.0346e-02,
         2.5579e-01,  3.4013e-01,  1.2085e-01, -3.6209e-01,  3.4075e-01,
        -6.5035e-02,  1.0430e-01,  4.8675e-01, -1.7171e-01, -5.7619e-02,
         1.0693e-01, -7.5097e-02,  5.4318e-02, -4.1089e-01, -4.0388e-01,
        -8.5236e-02, -1.2121e-02,  1.1091e-01, -2.9335e-01,  1.5991e-01,
         9.0246e-03,  1.4291e-01, -9.6933e-02,  2.6828e-01,  2.8086e-01,
         1.6056e-01,  2.6656e-01, -2.4157e-01, -1.3992e-01,  1.3189e-01,
         4.3679e-01, -2.7167e-02, -3.2675e-01, -1.

[tensor([ 1.2490e-01, -4.9502e-01,  5.6744e-01, -2.3659e-01,  3.8175e-01,
         4.3893e-01, -1.1877e-01,  3.7103e-01, -1.7097e-01, -1.9626e-01,
         1.0144e-01,  2.7507e-01, -3.6161e-01,  1.6815e-01,  9.0116e-02,
        -3.6048e-01,  1.1803e-01, -2.5103e-01, -3.9476e-01,  1.5905e-01,
        -9.4399e-02,  3.0480e-01,  5.9959e-03, -4.4267e-01, -6.2935e-02,
         6.2757e-01, -3.6357e-01, -2.1161e-01, -1.8411e-01, -1.6343e-01,
        -9.7723e-02,  1.8022e-02, -3.3463e-01,  1.3781e-01,  7.0072e-02,
        -9.1700e-02,  1.0467e-01,  9.9028e-02, -1.0364e-01,  2.3326e-01,
         1.6907e-01,  5.9418e-02, -3.7610e-02, -7.1423e-02, -1.0734e-01,
        -5.2044e-01, -1.5170e-01,  6.2913e-03, -4.8788e-01,  4.9032e-02,
         1.7364e-02, -6.8601e-02,  7.3822e-02, -1.2771e-02,  4.3408e-02,
        -9.5138e-02,  3.6122e-01, -8.3272e-02,  1.5772e-01,  2.3574e-01,
         1.5602e-01,  6.6783e-01, -2.0809e-01, -1.0011e-01, -8.0766e-02,
         1.7385e-01,  1.0808e-01, -3.8399e-01, -1.

In [50]:
from sentence_transformers import util
similarities_train = []
for i in range(len(sentence_embeddings_rp)):   
    similarities_train += [round(float(util.pytorch_cos_sim(sentence_embeddings_rp[i], sentence_embeddings_proj[i])[0]), 3)]
print(similarities_train)

[0.846, 0.838, 0.821, 0.882, 0.836, 0.834, 0.641, 0.804, 0.78, 0.777, 0.786, 0.839, 0.764, 0.758, 0.808, 0.821, 0.654, 0.637, 0.675, 0.715, 0.819, 0.806]


In [51]:
df_train_f1.drop('score', inplace=True, axis=1, errors='ignore')
df_train_f1.insert(loc=4, column='score', value=similarities_train)
df_train_f1

,id_rp,id_proj,name_rp,name_proj,score,text_rp,text_proj,need_match
9,985,8,Основы компьютерного дизайна,"Разработка ИС ""Расписание"" / ""Деканат"".",0.846,В курсе рассматриваются свойства зрительного в...,"Разработка ИС ""Расписание"" / ""Деканат"". Информ...",True
20,3524,57,Разработка защищенных программных систем,Стажировка в Digital Security по аудиту защищё...,0.838,Дисциплина посвящена изучению современных подх...,Стажировка в Digital Security по аудиту защищё...,True
2,1009,4,РАЗРАБОТКА ПРИЛОЖЕНИЙ В РАСПРЕДЕЛЕННОЙ СРЕДЕ,Разработка модулей обработки данных для библио...,0.821,Дисциплина посвящена изучению основных принцип...,Разработка модулей обработки данных для библио...,True
0,1000,15,АЛГОРИТМЫ БЕСПИЛОТНОГО ТРАНСПОРТА,Разработка системы движения роя дронов в прост...,0.882,Данный курс знакомит слушателей с основами упр...,Разработка системы движения роя дронов в прост...,True
13,4776,34,Web-технологии,Разработка модулей «Кафедра» и «Деканат-2» для...,0.836,Курс «Web-технологии» охватывает широкий спект...,Разработка модулей «Кафедра» и «Деканат-2» для...,True
16,1447,43,Моделирование нелинейных динамических систем,Нелинейная динамика и приложения теории хаоса,0.834,Дисциплина направлена на ознакомление студенто...,Нелинейная динамика и приложения теории хаоса....,True
10,998,45,Разработка в ядре Linux,Разработка САПР на основе платформы Beremiz дл...,0.641,дисциплина посвящена изучению проектирования к...,Разработка САПР на основе платформы Beremiz дл...,True
8,991,30,Машинное обучение,Разработка системы детектирования и идентифика...,0.804,Данный курс знакомит слушателей с основными ме...,Разработка системы детектирования и идентифика...,True
17,1098,56,Обработка изображений и компьютерное зрение (...,Практика в проектах по автономному транспорту ...,0.780,"Рассматриваются основные задачи, модели, метод...",Практика в проектах по автономному транспорту ...,True
7,1278,9,Тестирование программного обеспечения,Тестирование конфигураций на платформах: Super...,0.777,Дисциплина посвящена изучению теоретических ос...,Тестирование конфигураций на платформах: Super...,True


In [ ]:
# threshold = 0
# thresholds = []
# max_ = 0
# h = step
# steps = np.linspace(0, 1, num=int(1/h))
# steps = np.round(steps, 2)

# for i in steps:
#     for item in df_train_f1['text_rp']:
#         sentence_embeddings_rp.append(modelResearcher_rubert.model.encode(item, convert_to_tensor=True))

#     for item in df_train_f1['text_proj']:
#         sentence_embeddings_proj.append(modelResearcher_rubert.model.encode(item, convert_to_tensor=True))
        
#     sim = []
#     util.pytorch_cos_sim( sentences_1, sentences_2)
#     threshold = calc_f1_score(sim, df, h)
#     print(h, threshold)
#     thresholds.append(threshold)
#     if threshold > max_:
#         max_ = threshold
#     h += step
# plt.plot(steps, thresholds)
# plt.figure(figsize=(10,3))
# plt.show()